In [1]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

In [2]:
df = gpd.read_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/용인시_Grid_filtered_v5(app_cnt).geojson')
gas_df = pd.read_csv('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/신규등록주유소_utf8.csv',header=1)

In [3]:
gas_df.head()

,Unnamed: 0,연도,변동사유발생연월일,판매업종류,구분,업체명,소재지,위도,경도
0,652,2015,2015-08-11,주유소,신규등록,SK신갈주유소,경기도 용인시 기흥구 용구대로 2245,37.287621,127.107748
1,761,2015,2015-09-18,주유소,신규등록,옥토끼주유소,경기도 용인시 기흥구 용구대로 1991,37.265570,127.105069
2,1537,2016,2016-08-25,주유소,신규등록,서일석유(주)용인서일주유소,경기도 용인시 기흥구 신수로70(신갈동),37.276418,127.098534
3,1731,2016,2016-12-23,주유소,신규등록,포곡농협주유소,경기도 용인시 처인구 포곡읍 포곡로246번길 8,37.276443,127.229793
4,2024,2017,2017-05-24,주유소,신규등록,용인대주유소,경기도 용인시 처인구 이동면 서리 644-4외 1필지,37.209550,127.172570


In [4]:
# 주유소 반경 2km 처리
gas_df = gpd.GeoDataFrame(gas_df,geometry=gpd.points_from_xy(gas_df.경도, gas_df.위도),crs='epsg:4326').to_crs('epsg:5178')
gas_df['buffered']=gas_df.buffer(distance=2000)
gas_df.set_geometry('buffered',inplace=True)
gas_df.rename(columns={'geometry':'xy'},inplace=True)
gas_df.head()

,Unnamed: 0,연도,변동사유발생연월일,판매업종류,구분,업체명,소재지,위도,경도,xy,buffered
0,652,2015,2015-08-11,주유소,신규등록,SK신갈주유소,경기도 용인시 기흥구 용구대로 2245,37.287621,127.107748,POINT (965420.587 1920730.754),"POLYGON ((967420.587 1920730.754, 967410.957 1..."
1,761,2015,2015-09-18,주유소,신규등록,옥토끼주유소,경기도 용인시 기흥구 용구대로 1991,37.265570,127.105069,POINT (965172.977 1918285.422),"POLYGON ((967172.977 1918285.422, 967163.346 1..."
2,1537,2016,2016-08-25,주유소,신규등록,서일석유(주)용인서일주유소,경기도 용인시 기흥구 신수로70(신갈동),37.276418,127.098534,POINT (964598.574 1919491.306),"POLYGON ((966598.574 1919491.306, 966588.944 1..."
3,1731,2016,2016-12-23,주유소,신규등록,포곡농협주유소,경기도 용인시 처인구 포곡읍 포곡로246번길 8,37.276443,127.229793,POINT (976235.242 1919452.941),"POLYGON ((978235.242 1919452.941, 978225.611 1..."
4,2024,2017,2017-05-24,주유소,신규등록,용인대주유소,경기도 용인시 처인구 이동면 서리 644-4외 1필지,37.209550,127.172570,POINT (971136.668 1912047.804),"POLYGON ((973136.668 1912047.804, 973127.038 1..."


In [5]:
tqdm.pandas()
# Drop 할 polygon 안에 들어가는 points 에 drop column 추가
result = gpd.sjoin(df,gas_df,how='left',op='within')
# Drop table 과 join 안된 points 만 남기기
# result=result[result['주소'].isna()]
result=result.drop(columns=gas_df.drop(columns=['xy','buffered']).columns.values)
result=result.drop(columns=['index_right'])
result['xy']=result.progress_apply(lambda row: 0 if row.xy==None else row.xy,axis=1)
result['gas_dist']=result.progress_apply(lambda row: 0 if row.xy==0 else 1/(row.xy.distance(row.geometry)+1),axis=1)
result.drop(columns=['xy'],inplace=True)
result

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
100%|██████████| 15935/15935 [00:00<00:00, 39269.65it/s]


,gid,val,읍면동,ev_cnt,grid_bld_cnt,res_cnt,geometry,gas_dist
0,다사750101,0.0,이동읍,0.084702,2.0,0,POINT (975240.056 1909843.864),0.000000
1,다사914045,0.0,백암면,0.067067,2.0,0,POINT (991640.264 1904244.063),0.000000
2,다사918030,0.0,백암면,0.234735,7.0,0,POINT (992040.289 1902744.057),0.000000
3,다사595284,0.0,동천동,0.000000,0.0,0,POINT (959739.665 1928143.776),0.000000
4,다사720059,0.0,남사읍,1.332478,6.0,0,POINT (972240.096 1905643.788),0.000531
...,...,...,...,...,...,...,...,...
14284,다사631163,0.0,영덕2동,0.000000,0.0,0,POINT (963339.873 1916043.736),0.000762
14284,다사631163,0.0,영덕2동,0.000000,0.0,0,POINT (963339.873 1916043.736),0.000504
14285,다사581288,0.0,동천동,0.000000,0.0,0,POINT (958339.648 1928543.759),0.000000
14286,다사588268,0.0,신봉동,0.000000,0.0,0,POINT (959039.684 1926543.753),0.000000


In [6]:
result.to_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/용인시_Grid_filtered_v6(gas_dist).geojson')